## Determining the optimal number of hidden layers and neurons for an ANN

 This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

•	Start simple: Begin with simple architecture and gradually increase complexity if needed. 

•	Grid search / Random search: Use grid search or random search to try different architectures.

•	Cross- validation: Use cross- validation to evaluate the performance of different architectures. 

•	Heuristics and rules of thumb: Some heuristics and empirical rules can provide starting points, such as:

    o  	The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
    o	A common practice is to start with 1-2 hidden layers.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data= pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the Data
## 1. Drp Irreravlant Features

data= data.drop(['RowNumber','CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## 2. Label encode 'Gender' Column

le= LabelEncoder()
data['Gender']= le.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## 3. One Hot Encoding for 'Geography' Column

ohe= OneHotEncoder(handle_unknown='ignore')
geo_encoded= ohe.fit_transform(data[['Geography']]).toarray()

In [6]:
geo_encoded_df= pd.DataFrame(geo_encoded, columns= ohe.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [7]:
data= pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
# Dependent and Independent Features

X= data.drop('Exited', axis=1)
y= data['Exited']

In [9]:
# Train Test Split
X_train,X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=456)

In [10]:
stdscaler= StandardScaler()
X_train= stdscaler.fit_transform(X_train)
X_test= stdscaler.fit(X_test)

In [11]:
# Save the Pickle Files
with open('le.pkl', 'wb') as file:
    pickle.dump(le, file)

with open('ohe.pkl', 'wb') as file:
    pickle.dump(ohe, file)

with open('stdscaler.pkl', 'wb') as file:
    pickle.dump(stdscaler,file)

In [12]:
## Define a Function to create a model and try different parameters(KerasClassifier)
def create_model(neurons=32, layers=1):
    model= Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range (layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

### 🔁 Why use `_` in the loop?

- The underscore `_` is a **convention** used when the loop variable is not needed.
- In this case, we don't need the index — we only care about repeating an action a fixed number of times.
- Using `_` makes the code more concise and readable than writing `for i in range(...)` when `i` is not used.


### 🧱 Why `hidden_layers - 1`?

- When building a neural network, the **first hidden layer** is usually added separately because it requires the `input_dim` parameter.
- So, if `hidden_layers = 3`, we already added 1 manually, and we need to add **2 more layers**.
- That’s why we do:  
  ```python
  for _ in range(hidden_layers - 1):

In [13]:
model=KerasClassifier(neurons=32, layers=1, build_fn=create_model, epochs=50, batch_size=10, verbose=0)

C:\Users\harsu\AppData\Local\Temp\ipykernel_15052\4253886194.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(neurons=32, layers=1, build_fn=create_model, epochs=50, batch_size=10, verbose=0)


In [14]:
# Define the grid search Parameters

param_grid={
    'neurons':[16, 32, 64, 128],
    'layers': [1, 2],
    'epochs':[50,100]
}

In [15]:
# Perform Grid Search
grid= GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3, verbose=1)
grid_result= grid.fit(X_train, y_train)

# print Best Parameters
print("Best: %f using %s" %(grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best: 0.856875 using {'epochs': 50, 'layers': 1, 'neurons': 64}
